In [1]:
#import libraries

import plotly.graph_objects as go
import plotly.express as px
import os
import altair as alt
from vega_datasets import data
import pandas as pd
import hvplot
import matplotlib
import requests
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
import json

In [2]:
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")



In [4]:
# Instantiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    #base_url,
    api_version = "v2"
)

In [6]:
# Get user input for the stock symbol
# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")
stock_symbol = "META"

In [5]:
# Establish what time frame increment we will be looking at
time_frame = "1Day"

In [7]:
# Get user input for the start date in datetime format
# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2020-10-10", tz="America/New_York").isoformat()


In [8]:
# Get user input for the end date in datetime format
# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-12-12", tz="America/New_York").isoformat()

In [9]:
# trading_days = end_date - start_date


# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [9]:
# Calls the stock data from the period of time desired in the desired time increment
stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [10]:
# Displays the information pulled
stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44 entries, 2020-10-12 04:00:00+00:00 to 2020-12-11 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        44 non-null     float64
 1   high         44 non-null     float64
 2   low          44 non-null     float64
 3   trade_count  44 non-null     int64  
 4   open         44 non-null     float64
 5   volume       44 non-null     int64  
 6   vwap         44 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 2.8 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,292730,270.200,32999660,275.227035
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,181923,277.580,19505211,276.201124
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,151136,277.620,16590158,273.981850
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,182305,267.600,16330684,266.033730
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,157636,267.375,18430693,267.458035


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-12-07 05:00:00+00:00,285.58,288.4900,278.20,153281,279.190,14005842,285.751969
2020-12-08 05:00:00+00:00,283.40,286.4272,281.55,136454,286.010,11974657,283.705184
2020-12-09 05:00:00+00:00,277.92,287.6300,271.75,312698,283.660,26765687,278.642389
2020-12-10 05:00:00+00:00,277.12,278.7300,271.86,209825,275.535,21534833,276.018129
2020-12-11 05:00:00+00:00,273.55,276.4800,270.25,158853,274.530,15609459,273.330511


In [12]:
# Add new data columns to DF for body size and candle size
# body size is absolute value of open - close
# candle size is the absolute value of low - high
# clean up datafram by removing vwap, volume, trade count columns


In [11]:
stock_data = stock_data.reset_index()

In [12]:
stock_data.head()

,timestamp,close,high,low,trade_count,open,volume,vwap
0,2020-10-12 04:00:00+00:00,275.75,280.18,267.87,292730,270.200,32999660,275.227035
1,2020-10-13 04:00:00+00:00,276.14,279.10,273.39,181923,277.580,19505211,276.201124
2,2020-10-14 04:00:00+00:00,271.82,278.75,271.50,151136,277.620,16590158,273.981850
3,2020-10-15 04:00:00+00:00,266.72,269.04,263.67,182305,267.600,16330684,266.033730
4,2020-10-16 04:00:00+00:00,265.93,271.37,265.30,157636,267.375,18430693,267.458035


In [13]:
fig = go.Figure([go.Scatter(x=stock_data['timestamp'], y=stock_data['close'])])

In [14]:
fig

In [15]:
fig = go.Candlestick(x=stock_data['timestamp'],
                open=stock_data['open'],
                high=stock_data['high'],
                low=stock_data['low'],
                close=stock_data['close'])


In [16]:
cand = go.Figure(data=[fig])

In [17]:
cand.show()

In [ ]:
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results
stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
display(stock_data.head())

In [ ]:
new_data = stock_data.reset_index(inplace=True)

In [ ]:
print(new_data)

In [ ]:
stock_data.head(2)

In [ ]:
fig = go.Figure(stock_data=[go.Candlestick(x=stock_data['timestamp'],
                open=stock_data['META.Open'],
                high=stock_data['META.High'],
                low=stock_data['META.Low'],
                close=stock_data['META.Close'])])


In [ ]:
fig = px.candlestick(stock_data, x='timestamp', open='open', high='high', low='low', close='close')

# Customize the figure (optional)
fig.update_layout(title="Candlestick Chart Example", xaxis_title="Date", yaxis_title="Price")
fig.update_xaxes(type='category')



In [ ]:
# Show the chart
fig.show()

In [ ]:
chart = alt.Chart(stock_data).mark_area().encode(
    x="close",
    y="open"
)

In [ ]:
chart

In [ ]:
# create line plot
chart = alt.Chart(stock_data).mark_line().encode(x="timestamp",y="Close")
# set title and axis lables
chart.title = "META stock closing prices"
chart.encoding.x.title = "Time"
chart.encoding.y.title = "Price [USD]"


In [ ]:
base = alt.Chart(stock_data)

In [ ]:
candlesticks = base.mark_bar().encode(
    x='timestamp:T',
    y='low:Q',
    y2='high:Q',
    color=alt.condition("datum.open <= datum.close", alt.value("green"), alt.value("red")),
    tooltip=['open:Q', 'high:Q', 'low:Q', 'close:Q']
)

In [ ]:
wicks = base.mark_rule().encode(
    x='timestamp:T',
    y='low:Q',
    y2='high:Q'
)


In [ ]:
chart = candlesticks + wicks

In [ ]:
chart

In [ ]:
# display line plot
chart

In [ ]:
# Add new data columns to DF for body size and candle size
# body size is absolute value of open - close
# candle size is the absolute value of low - high
# clean up datafram by removing vwap, volume, trade count columns

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

In [ ]:
stock_data.head()

In [ ]:
source = stock_data

open_close_color = alt.condition(
    "datum.open <= datum.close",
    alt.value("#06982d"),
    alt.value("#ae1325")
)

base = alt.Chart(source).encode(
    alt.X('date:T')
        .axis(format='%m/%d', labelAngle=-45)
        .title('Date in 2020'),
    color=open_close_color
)

rule = base.mark_rule().encode(
    alt.Y('low:Q')
        .title('Price')
        .scale(zero=False),
    alt.Y2('high:Q')
)

bar = base.mark_bar().encode(
    alt.Y('open:Q'),
    alt.Y2('close:Q')
)

rule + bar

In [ ]:
stock_data['body_size'].head()

In [ ]:
max_body_size = stock_data["body_size"].max()

In [ ]:
min_candle_size = stock_data["candle_size"].min()

In [ ]:
display(max_body_size)
display(min_candle_size)

In [ ]:
stock_data_1 = stock_data.drop(columns=['high','low','body_size','candle_size'])

In [ ]:
display (stock_data_1.head())

In [ ]:
current_count = 0

In [ ]:
if stock_data.iloc[0][4] > 0:
    previous_candle = 1
elif stock_data.iloc[0][4] == 0:
    previous_candle = 0
else: stock_data.iloc[0][4] < 0
previous_candle = -1
 
number_of_rows = len(stock_data)

display(previous_candle)
display(stock_data.iloc[0][4])
display(number_of_rows)

In [ ]:
row_counter = 0

for row in stock_data:
    row_counter = row_counter + 1
    if stock_data.iloc[row_counterr][3] < stock_data.iloc[row_counter][0] and ads(st
        